# LightBox API - Purpose of Condo Logic

Use an address to acquire its location coordinates, then use those coordinates to find an associated property parcel, then use that parcel to find associated tax assessment data.

# LightBox API - Using Addresses to Find Tax Assessment Data

This notebook demonstrates basic features of the LightBox Geocoding API by walking through the following steps:

1. Import Python packages
2. Enter your **API Key** for authorization
3. Create request objects and display the results both in the JSON form.

Additional Materials:
[LightBox Developer Portal](https://developer.lightboxre.com/)

#1. Import the required python packages

In [8]:
import requests
import json
from typing import Dict
import pandas as pd


#2. Import function definitions

In [9]:
# ----------------------------
# Function Definitions
# ----------------------------

# Function to geocode a single address using the LightBox API.
def geocode_address(lightbox_api_key: str, address: str) -> Dict:
    """
    Geocodes the provided address using the LightBox API.
    
    Args:
        lightbox_api_key (str): The API key for accessing the LightBox API.
        address (str): The address string for matching.
    
    Returns:
        dict: The geocoded address information in JSON format.
    """
    # API endpoint configuration
    BASE_URL = "https://api.lightboxre.com/v1"
    ENDPOINT = "/addresses/search"
    URL = BASE_URL + ENDPOINT

    # Setting up request parameters and headers
    params = {"text": address}
    headers = {"x-api-key": lightbox_api_key}

    # Sending request to the LightBox API
    geocoder_data = requests.get(URL, params=params, headers=headers)
    
    # Returning the geocoded address information
    return geocoder_data


# Test for the get_parcel_from_lbx_address_id() function
def get_parcel_data_from_address_coordinates(lightbox_api_key: str, country_code: str, address_wkt_coordinates: str) -> Dict:
    """
    Returns a dictionary containing the parcel data for the specified address.

    Args:
        lightbox_api_key (str): The API key for accessing the LightBox API.
        country_code (str): The country code for the address.
        address_wkt_coordinates (str): The address coordinates for the address.
    """

    # API endpoint configuration
    BASE_URL = "https://api.lightboxre.com/v1"
    ENDPOINT = f"/parcels/{country_code}/geometry"
    URL = BASE_URL + ENDPOINT

    # Setting up request parameters and headers
    params = {"wkt": address_wkt_coordinates}
    headers = {"x-api-key": lightbox_api_key}

    # Make the request
    parcel_data = requests.get(URL, params=params, headers=headers)
    
    # Return the parcel data
    return parcel_data


def get_assessment_data_from_lbx_parcel_id(lightbox_api_key: str, parcel_id: str) -> Dict:
    """
    Returns a dictionary containing the assessment data for the specified parcel.

    Args:
        lightbox_api_key (str): The LightBox API key.
        parcel_id (str): The parcel ID.
    """

    # API endpoint configuration
    BASE_URL = "https://api.lightboxre.com/v1"
    ENDPOINT = f"/assessments/_on/parcel/us/{parcel_id}"
    URL = BASE_URL + ENDPOINT

    # Setting up request parameters and headers
    headers = {"x-api-key": lightbox_api_key}

    # Make the request
    assessment_data = requests.get(URL, headers=headers)
    
    # Return the assessment data
    return assessment_data


#3. Create variables that will be used to authenticate your calls.
Get your key from the [LightBox Developer Portal](https://developer.lightboxre.com/).

In [13]:
lightbox_api_key = '<YOUR_API_KEY>'

#4. LightBox API Details
This notbook will create various requests and display the output in JSON.
1. **Geocoding** - https://api.lightboxre.com/v1/addresses/search
2. **Parcels** - https://api.lightboxre.com/v1/parcels/{country_code}/geometry
3. **Assessments** - https://api.lightboxre.com/v1/assessments/_on/parcel/us/{lbx_parcel_id}

For additional details regarding each endpoint's request parameters or response models, visit the [API Catalog](https://developer.lightboxre.com/apis) on the LightBox Developer Portal page.

Create variables containing necessary inputs for subsequent calls.


1. Notice how the API returns any addresses within the buffer distance of the provided geometry, as well as any metadata associated with that address

In [14]:
# ----------------------------
# API Usage
# ----------------------------

# Specify the address to geocode
address = "25482 Buckwood Land Forest, Ca, 92630"
country_code = "us" # 'us' for the United States

# Geocode the specified address
address_search_data = geocode_address(lightbox_api_key, address)

# Get the parcel data from the geocoded address
parcel_data = get_parcel_data_from_address_coordinates(lightbox_api_key, country_code, address_search_data.json()["addresses"][0]["location"]["representativePoint"]["geometry"]["wkt"])

# Get the assessment data from the parcel ID
assessment_data = get_assessment_data_from_lbx_parcel_id(lightbox_api_key, parcel_data.json()["parcels"][0]["id"])


In [29]:
pd.json_normalize(address_search_data.json()["addresses"])

,$ref,id,uuaid,uaid,label,type,addressConfidenceScore,postalCodeDerived,corroborationIndex,parcels,...,location.locality,location.regionCode,location.unit,location.county,location.countryCode,location.postalCode,location.postalCodeExt,location.representativePoint.latitude,location.representativePoint.longitude,location.representativePoint.geometry.wkt
0,https://api.lightboxre.com/v1/addresses/search...,06033UO1O5JI8WZTQ8O4SG,None,None,"25482 BUCKWOOD, LAKE FOREST, CA 92630-5433, US",None,None,None,None,[{'$ref': 'https://api.lightboxre.com/v1/parce...,...,LAKE FOREST,CA,None,ORANGE COUNTY,US,92630,5433,33.62776,-117.685771,POINT(-117.685771 33.62776)


In [25]:
pd.json_normalize(parcel_data.json()["parcels"])


,$ref,id,fips,parcelApn,county,legalDescription,neighborhood,opportunityZone,subdivision,structures,...,transaction.lastMarketSale.tdDocumentNumber,transaction.lastMarketSale.deedTransactionType,transaction.lastMarketSale.lenderType,transaction.priorMarketSale.transferDate,transaction.priorMarketSale.lender,transaction.multipleApnFlag,usplss.township,usplss.range,usplss.section,usplss.quarter
0,https://api.lightboxre.com/v1/parcels/us/02011...,02011302EQSMZB5I2YM3IV,06059,613-211-09,ORANGE,[N TR 10261 BLK LOT 9],None,False,None,[],...,None,2,M,None,,A,None,None,None,None


In [26]:
pd.json_normalize(assessment_data.json()["assessments"])


,$ref,apn,fips,id,alternateApn,improvementPercent,assessedLotSize,poolIndicator,book,page,...,tax.amount,tax.rateAreaCode,tax.exemption,valuationModel.value,valuationModel.valueHigh,valuationModel.valueLow,valuationModel.date,valuationModel.propensityScore,valuationModel.score,valuationModel.label
0,https://api.lightboxre.com/v1/assessments/us/0...,613-211-09,06059,0307LTEIHJOJRD1XGXP84X,None,36.95,483.0956,S,31080,0605991977,...,3876.0,30-045,None,1062000.0,1178820.0,945180.0,None,None,89.0,$1.06 M


In [27]:
# --------------------
# Print collected data
# --------------------

# Print the geocoded address data in a readable JSON format
print(json.dumps(address_search_data.json(), indent=4))
# Print the parcel data in a readable JSON format
print(json.dumps(parcel_data.json(), indent=4))
# Print the assessment data in a readable JSON format
print(json.dumps(assessment_data.json(), indent=4))



{
    "$ref": "https://api.lightboxre.com/v1/addresses/search?text=25482%20Buckwood%20Land%20Forest%2C%20Ca%2C%2092630",
    "$metadata": {
        "geogcs": {
            "epsg": "4326"
        },
        "recordSet": {
            "totalRecords": 1.0
        }
    },
    "addresses": [
        {
            "$ref": "https://api.lightboxre.com/v1/addresses/search?text=25482%20Buckwood%2C%20Lake%20Forest%2C%20CA%2092630-5433%2C%20USA",
            "id": "06033UO1O5JI8WZTQ8O4SG",
            "uuaid": null,
            "uaid": null,
            "$metadata": {
                "geocode": {
                    "confidence": {
                        "streetNumber": 1,
                        "streetName": 0.9230769230769231,
                        "locality": 1,
                        "postalCode": 1,
                        "score": 0.949
                    },
                    "precisionCode": 10,
                    "changeFlag": {
                        "streetName": "Fuzzy",
    

# Commonly Seen Errors

See [LightBox Geocoder API](https://developer.lightboxre.com/api/geocoding#/Autocomplete/get_addresses__autocomplete) for a list of common error responses. 